In [1]:
%pip install tensorflow --quiet

Note: you may need to restart the kernel to use updated packages.


#For creating the model that train the data from scratch, we need:
1. Layer
2. Sequential Class
3. Epoch
4. Batch
5. Datae

In [12]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [13]:
import math
class BatchGenerator:
    def __init__(self, images, labels, batch_size = 128):
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index: self.index + self.batch_size]
        labels = self.labels[self.index: self.index + self.batch_size]
        self.index+= self.batch_size
        return images, labels

In [14]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        print(per_sample_losses)
        average_loss = tf.reduce_mean(per_sample_losses)
        print(average_loss)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [15]:
learning_rate = 1e-3
def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g* learning_rate)